In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import time

from selenium import webdriver

In [ ]:
years = list(range(1991, 2023))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

# Scrape MVPs


In [ ]:
dfs = []

# Already fetched
# for year in years:
#   url = url_start.format(year)
#   data = requests.get(url)
#   file_name = "./mvps/{}.html".format(year)


#   if os.path.exists(file_name):
#     os.remove(file_name)
#   with open(file_name, "w+") as f:
#     f.write(data.text)

In [ ]:
for year in years:
    with open("./mvps/{}.html".format(year)) as f:
        page = f.read()

    soup = bs(page, "html.parser")
    soup.find("tr", class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")
    mvp = pd.read_html(str(mvp_table))[0]
    mvp["Year"] = year

    print(year)

    dfs.append(mvp)
    mvps = pd.concat(dfs)

In [ ]:
if os.path.exists("mvps.csv"):
    os.remove("mvps.csv")
mvps.to_csv("mvps.csv")

# Scrape player stats


In [ ]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [ ]:
# options = webdriver.FirefoxOptions()
# driver = webdriver.Remote(command_executor=os.environ["SELENIUM_URL"], options=options)

# for year in years:
#     url = player_stats_url.format(year)
#     print(f"scraping {year} data")

#     driver.get(url)
#     driver.execute_script("window.scrollTo(1,10000)")
#     time.sleep(2)

#     with open("player/{}.html".format(year), "w+") as f:
#         f.write(driver.page_source)

# driver.close()

In [ ]:
dfs = []

for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()

    soup = bs(page, "html.parser")
    soup.find("tr", class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

players = pd.concat(dfs)
players.to_csv("players.csv")


# Team Stats

In [ ]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [ ]:
for year in years:
    url = team_stats_url.format(year)
    
    data = requests.get(url)
    time.sleep(2)
    
    with open("team/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [ ]:
dfs = []
for year in years:
    with open("team/{}.html".format(year)) as f:
        page = f.read()
    
    soup = bs(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    e_table = soup.find_all(id="divs_standings_E")[0]
    e_df = pd.read_html(str(e_table))[0]
    e_df["Year"] = year
    e_df["Team"] = e_df["Eastern Conference"]
    del e_df["Eastern Conference"]
    for i in range(len(e_df["Team"])):
        e_df["Team"][i] = e_df["Team"][i].split("*")[0]

    dfs.append(e_df)
    
    w_table = soup.find_all(id="divs_standings_W")[0]
    w_df = pd.read_html(str(w_table))[0]
    w_df["Year"] = year
    w_df["Team"] = w_df["Western Conference"]
    del w_df["Western Conference"]
    for i in range(len(w_df["Team"])):
        w_df["Team"][i] = w_df["Team"][i].split("*")[0]
    dfs.append(w_df)

In [ ]:
teams = pd.concat(dfs)
teams

In [ ]:
teams.to_csv("teams.csv")

# Name abbreviations

In [ ]:
url = "https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations"

data = requests.get(url)
with open("abb/name.html", "w+") as f:
  f.write(data.text)
data.close()

In [ ]:
with open("abb/name.html") as f:
  page = f.read()
soup = bs(page, 'html.parser')
table = soup.find_all("table", class_="wikitable sortable")[0]
[sup.decompose() for sup in soup("sup")]
abrev_df = pd.read_html(str(table))[0]

for i in range(len(abrev_df['Abbreviation/ Acronym'])):
  abrev_df['Abbreviation/ Acronym'][i] = abrev_df['Abbreviation/ Acronym'][i].split(' ')[0]

abrev_df.to_csv("abrev.csv")